# Audiobooks business case

## Preprocess the data. Balance the dataset. Standardize the data. Create 3 datasets: training, validation, and test. Save the newly created sets in a tensor friendly format (e.g. *.npz)

Since we are dealing with real life data, we will need to preprocess it a bit.

If you want to know how to do that, go through the code with comments. In any case, this should do the trick for most datasets organized in the way: many inputs, and then 1 cell containing the targets (supersized learning datasets). Keep in mind that a specific problem may require additional preprocessing.

Note that we have removed the header row, which contains the names of the categories. We simply need the numerical data.

### Extract the data from the csv

In [13]:
import numpy as np

# We will use the StandardScaler module, so we can later deploy the model
from sklearn.preprocessing import StandardScaler

import pickle

import tensorflow as tf


In [21]:
from tensorflow.keras.models import Sequential


In [17]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Load the data
raw_csv_data = np.loadtxt('Audiobooks_data.csv',delimiter=',')

# The inputs are all columns in the csv, except for the first one and the last one
# The first column is the arbitrary ID, while the last contains the targets

unscaled_inputs_all = raw_csv_data[:,1:-1]

# The targets are in the last column. That's how datasets are conventionally organized.
targets_all = raw_csv_data[:,-1]

### Balance the dataset

In [2]:
# There are different Python packages that could be used for balancing
# Here we approach the problem manually, so you can observe the inner workings of the balancing process

# Count how many targets are 1 (meaning that the customer did convert)
num_one_targets = int(np.sum(targets_all))

# Set a counter for targets that are 0 (meaning that the customer did not convert)
zero_targets_counter = 0

# We want to create a "balanced" dataset, so we will have to remove some input/target pairs.
# Declare a variable that will do that:
indices_to_remove = []

# Count the number of targets that are 0. 
# Once there are as many 0s as 1s, mark entries where the target is 0.
for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)

# Create two new variables, one that will contain the inputs, and one that will contain the targets.
# We delete all indices that we marked "to remove" in the loop above.
unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis=0)

### Standardize the inputs

In [8]:
# Crete a standar scaler object
scaler_deep_learning = StandardScaler()
# Fit and transform the original data
# Essentially, we calculate and STORE the mean and variance of the data in the scaler object
# At the same time we standrdize the data using this information
# Note that the mean and variance remain recorded in the scaler object
scaled_inputs = scaler_deep_learning.fit_transform(unscaled_inputs_equal_priors)

### Shuffle the data

In [9]:
# When the data was collected it was actually arranged by date
# Shuffle the indices of the data, so the data is not arranged in any way when we feed it.
# Since we will be batching, we want the data to be as randomly spread out as possible
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

# Use the shuffled indices to shuffle the inputs and targets.
shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

### Split the dataset into train, validation, and test

In [10]:
# Count the total number of samples
samples_count = shuffled_inputs.shape[0]

# Count the samples in each subset, assuming we want 80-10-10 distribution of training, validation, and test.
# Naturally, the numbers are integers.
train_samples_count = int(0.8 * samples_count)
validation_samples_count = int(0.1 * samples_count)

# The 'test' dataset contains all remaining data.
test_samples_count = samples_count - train_samples_count - validation_samples_count

# Create variables that record the inputs and targets for training
# In our shuffled dataset, they are the first "train_samples_count" observations
train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

# Create variables that record the inputs and targets for validation.
# They are the next "validation_samples_count" observations, folllowing the "train_samples_count" we already assigned
validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

# Create variables that record the inputs and targets for test.
# They are everything that is remaining.
test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

# We balanced our dataset to be 50-50 (for targets 0 and 1), but the training, validation, and test were 
# taken from a shuffled dataset. Check if they are balanced, too. Note that each time you rerun this code, 
# you will get different values, as each time they are shuffled randomly.
# Normally you preprocess ONCE, so you need not rerun this code once it is done.
# If you rerun this whole sheet, the npzs will be overwritten with your newly preprocessed data.

# Print the number of targets that are 1s, the total number of samples, and the proportion for training, validation, and test.
print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

1773.0 3579 0.4953897736797988
229.0 447 0.5123042505592841
235.0 448 0.5245535714285714


### Save the three datasets in *.npz

In [11]:
# Save the three datasets in *.npz.
# In the next lesson, you will see that it is extremely valuable to name them in such a coherent way!

np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Audiobooks_data_test', inputs=test_inputs, targets=test_targets)

### Save the scaler

In [12]:
# Similar to how we have saved the scaler files before, we also save this scaler, so we can apply in on new data
pickle.dump(scaler_deep_learning, open('scaler_deep_learning.pickle', 'wb'))

### Deep learning model. 

#### It has 10 input units which are the inputs from the csv and 2 output units which are the 2 categories: if a person purchase again(1) or not(0)

### Data.

In [18]:
npz_train = np.load('Audiobooks_data_train.npz')
train_inputs = npz_train['inputs'].astype(np.float)
train_targets = npz_train['targets'].astype(np.int)


npz_validation = np.load('Audiobooks_data_validation.npz')
validation_inputs = npz_validation['inputs'].astype(np.float)
validation_targets = npz_validation['targets'].astype(np.int)

npz_test = np.load('Audiobooks_data_test.npz')
test_inputs = npz_test['inputs'].astype(np.float)
test_targets = npz_test['targets'].astype(np.int)


### Model. 

In [22]:
#input_size = 10
output_size = 2
hidden_layer_size = 50

model = tf.keras.Sequential([
                            tf.keras.layers.Dense(hidden_layer_size,activation = 'relu'),
                            tf.keras.layers.Dense(hidden_layer_size,activation = 'relu'),
                            tf.keras.layers.Dense(output_size,activation = 'softmax')
                            ])

model.compile(optimizer = 'adam',loss = 'sparse_categorical_crossentropy',metrics = ['accuracy'])

batch_size = 100

max_epochs = 100

early_stopping = tf.keras.callbacks.EarlyStopping(patience = 2) # prevent the overfitting. 

model.fit(train_inputs,
          train_targets,
          batch_size = batch_size,
          epochs = max_epochs, 
          callbacks = [early_stopping],
          validation_data = (validation_inputs,validation_targets),
          verbose = 2
        )

Epoch 1/100
36/36 - 3s - loss: 0.6039 - accuracy: 0.7415 - val_loss: 0.4652 - val_accuracy: 0.8658
Epoch 2/100
36/36 - 0s - loss: 0.3976 - accuracy: 0.8726 - val_loss: 0.3311 - val_accuracy: 0.8881
Epoch 3/100
36/36 - 0s - loss: 0.3283 - accuracy: 0.8790 - val_loss: 0.3078 - val_accuracy: 0.8904
Epoch 4/100
36/36 - 0s - loss: 0.3059 - accuracy: 0.8871 - val_loss: 0.2909 - val_accuracy: 0.8971
Epoch 5/100
36/36 - 0s - loss: 0.2911 - accuracy: 0.8902 - val_loss: 0.2840 - val_accuracy: 0.8949
Epoch 6/100
36/36 - 0s - loss: 0.2805 - accuracy: 0.8949 - val_loss: 0.2789 - val_accuracy: 0.8949
Epoch 7/100
36/36 - 0s - loss: 0.2734 - accuracy: 0.8969 - val_loss: 0.2776 - val_accuracy: 0.8904
Epoch 8/100
36/36 - 0s - loss: 0.2648 - accuracy: 0.9003 - val_loss: 0.2689 - val_accuracy: 0.8971
Epoch 9/100
36/36 - 0s - loss: 0.2596 - accuracy: 0.9011 - val_loss: 0.2670 - val_accuracy: 0.9038
Epoch 10/100
36/36 - 0s - loss: 0.2551 - accuracy: 0.9014 - val_loss: 0.2675 - val_accuracy: 0.8993
Epoch 11/

### Val accuracy 93%. In 93% of the cases our model has predicted whether a customer will convert again. 

### Test the model. 

In [23]:
test_loss, test_accuracy = model.evaluate(test_inputs,test_targets)

14/14 [==============================] - 0s 2ms/step - loss: 0.2399 - accuracy: 0.9107


### Obtain the probability for a customer to convert. 

In [25]:
model.predict(test_inputs).round(2)

array([[0.12, 0.88],
       [0.  , 1.  ],
       [0.99, 0.01],
       [0.02, 0.98],
       [0.9 , 0.1 ],
       [0.94, 0.06],
       [0.01, 0.99],
       [0.85, 0.15],
       [0.23, 0.77],
       [0.  , 1.  ],
       [0.97, 0.03],
       [0.  , 1.  ],
       [0.95, 0.05],
       [0.33, 0.67],
       [0.91, 0.09],
       [0.3 , 0.7 ],
       [0.95, 0.05],
       [1.  , 0.  ],
       [0.16, 0.84],
       [0.08, 0.92],
       [0.93, 0.07],
       [0.88, 0.12],
       [0.15, 0.85],
       [0.28, 0.72],
       [0.94, 0.06],
       [0.94, 0.06],
       [0.34, 0.66],
       [0.11, 0.89],
       [0.78, 0.22],
       [0.01, 0.99],
       [0.  , 1.  ],
       [0.01, 0.99],
       [0.16, 0.84],
       [0.85, 0.15],
       [0.94, 0.06],
       [0.  , 1.  ],
       [0.3 , 0.7 ],
       [0.44, 0.56],
       [0.  , 1.  ],
       [0.04, 0.96],
       [0.95, 0.05],
       [0.  , 1.  ],
       [0.16, 0.84],
       [1.  , 0.  ],
       [0.23, 0.77],
       [0.03, 0.97],
       [0.76, 0.24],
       [0.14,

### Each row represents a different observation.
#### The first column is the probability to be classified as 0 and the second column the probability to be classified as 1.  

In [26]:
model.predict(test_inputs)[:,1].round(0)

array([1., 1., 0., 1., 0., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 1., 0.,
       0., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0.,
       0., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1., 0., 1., 0.,
       1., 0., 0., 0., 1., 1., 1., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0.,
       1., 0., 1., 1., 1., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0.,
       1., 0., 1., 0., 0., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1., 0., 0.,
       1., 1., 0., 1., 1., 1., 1., 0., 1., 0., 1., 1., 0., 0., 1., 1., 0.,
       0., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 0., 1.,
       0., 1., 0., 1., 1., 1., 1., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0.,
       1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 1., 0., 1., 1., 1.,
       0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0.,
       0., 0., 1., 0., 1., 0., 1., 1., 1., 1., 0., 1., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 0., 1., 0., 1., 0., 0.,
       1., 0., 0., 0., 1.

### For multiclass approach use argmax 

### Axis = 1 for columns. 


In [27]:
np.argmax(model.predict(test_inputs),axis=1)

array([1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0,
       1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0,
       1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1,
       1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0,
       1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1,
       1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1,
       0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1,

### Hierarchical data format. (H5) 

In [28]:
model.save('audiobooks_model.h5')

## **Predict on new data**

### Load the scaler and the model. 

In [29]:
scaler_deep_learning = pickle.load(open('scaler_deep_learning.pickle','rb'))
model = tf.keras.models.load_model('audiobooks_model.h5')

In [30]:
raw_data = np.loadtxt('New_Audiobooks_Data.csv',delimiter = ',' )

In [31]:
new_data_inputs = raw_data[:,1:]

In [33]:
new_data_inputs_scaled = scaler_deep_learning.transform(new_data_inputs)

In [35]:
np.argmax(model.predict(new_data_inputs_scaled),1)

array([0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 1], dtype=int64)